In [14]:
from torchvision.models import resnet34
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch



https://www.reddit.com/r/MachineLearning/comments/kvs1ex/d_here_are_17_ways_of_making_pytorch_training/
Tried some of the trick in this post for 1 epoch. ~17 seconds is with Cuda on my GPU (RTX 3070) 
Defining num_workers and pin_memory made the performance worse with resnet34, may be different with other models.
Same with torch.backends.cudnn.benchmark = True
Increasing the batch_size to 128 sometimes increased the performance from ~17secs -> ~16 but not always

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = ImageFolder("./Colorectal Cancer", transform=ToTensor())
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])
train_loader = DataLoader(train_set, shuffle=True, batch_size=64)
validation_loader = DataLoader(validation_set, shuffle=True, batch_size=64)
test_loader = DataLoader(test_set, shuffle=True, batch_size=64)
print("Data loaded")

Data loaded


In [16]:

model = resnet34(num_classes=3).to(device)
torch.manual_seed(0)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)
loss = torch.nn.CrossEntropyLoss()

num_epoch = 1

In [17]:
scaler = torch.cuda.amp.GradScaler()
for epoch in range(num_epoch):
    
    # Pass an epoch over the training data in batch_size chunks
    for features, labels in train_loader:
        features = features.to(device)
        labels = labels.to(device)

        # forward
        y_pred = model(features)
        l = loss(y_pred, labels)
        model.zero_grad()
        # backprop and step
        l.backward()
        optimizer.step()
        print(f"minibatch loss: {l}")
    print(f"Epoch {epoch}: last batch loss: {l}")

minibatch loss: 1.097245693206787
minibatch loss: 1.1632064580917358
minibatch loss: 1.0653982162475586
minibatch loss: 1.2783136367797852
minibatch loss: 1.195480465888977
minibatch loss: 1.1551282405853271
minibatch loss: 1.0241748094558716
minibatch loss: 0.864782989025116
minibatch loss: 0.7977977395057678
minibatch loss: 0.8401590585708618
minibatch loss: 0.8976565599441528
minibatch loss: 0.8312989473342896
minibatch loss: 0.8491672873497009
minibatch loss: 0.9039831757545471
minibatch loss: 0.7387800812721252
minibatch loss: 0.9395275712013245
minibatch loss: 0.8093081712722778
minibatch loss: 0.9066091179847717
minibatch loss: 0.7404758930206299
minibatch loss: 0.800789475440979
minibatch loss: 0.756416916847229
minibatch loss: 0.9358212947845459
minibatch loss: 0.6864200830459595
minibatch loss: 0.48586544394493103
minibatch loss: 0.768983006477356
minibatch loss: 1.3711438179016113
minibatch loss: 0.525038480758667
minibatch loss: 0.6649064421653748
minibatch loss: 0.62874424

In [18]:
# torch.cuda.empty_cache()